In [11]:
# What are the variables we want set as default?
# Taking out the user input leaves much less comparison options? what do we want to compare actually?
# Now bin_width=500, wanted_bin=0 occure as kew words in main() and in binning() function. Is this OK?
# Do we want to have global variable declarations in the header?

import pyspeckit as psk
from spectral_cube import SpectralCube
import astropy.units as u

import GAS
from spectral_cube import SpectralCube
import numpy as np
import matplotlib.pyplot as plt
import scipy.ndimage as nd
import astropy.units as u
from astropy.io import fits
from astropy.table import Table

import pyspeckit as psk
from pyspeckit.spectrum.models import ammonia
from pyspeckit.spectrum.models import ammonia_constants, ammonia, ammonia_hf
from pyspeckit.spectrum.models.ammonia_constants import freq_dict
from pyspeckit.spectrum.units import SpectroscopicAxis, SpectroscopicAxes

%pylab
%matplotlib inline

vlsr = fits.getdata('NGC1333_Vlsr_v1.fits')
files =['NGC1333_NH3_11_DR1.fits', 'NGC1333_NH3_22_DR1.fits', 'NGC1333_NH3_33_DR1.fits', 
       'NGC1333_C2S_DR1.fits',
       'NGC1333_HC5N_DR1.fits',
        'NGC1333_HC7N_21_20_DR1.fits', 'NGC1333_HC7N_22_21_DR1.fits']


# lines_margins = {files[0]: [[-1.5, 3.],[6.25, 11.],[18.5,22.5], [-8.5,-4.5], [-21., -16.5]], files[1]: [[-2., 3.], [-17., -13.5], [15.5, 19.]],
#                  files[2]: [[-4.25, 4]], files[3]: [[-1.5, 3.]], files[4]: [[-2, 4.]], files[5]: [[-1.5, 3.]],
#                     files[6]: [[-1.5, 3.]]}
# noise_margins = {files[0]: [12., 17.], files[1]: [5., 12.5], files[2]: [-12, -10], files[3]: [-12, -10], files[4]: [10., 27.5], files[5]: [5., 15.],
#                     files[6]: [5., 15.]}

# bin_width = 500
# this_bin = 69
# bin_num = 50
map_temp = 'NGC1333_Temperature.fits'
map_name = 'NGC1333_DustColumn.fits'
# map_binning = 'OrionA_NH3_11_base_DR1_mom0.fits'

def main(map_name):   
    # This is the main routine.
    Tmax = fits.getdata(map_name)
    bin_arr = np.sort(Tmax[np.isfinite(Tmax)])[::-1]
    max_len = len(bin_arr)

    y, x, med, bin_num = progression_binning(max_len, map_name)    
    t, table_names = make_table_and_plot(files, map_name, y, x, table_content = 'chemical_abundance')
    t
    
#     plot_spectra(this_bin, bin_width, map_name)
#     spec_curve_fit(this_bin, map_name)
#     t, table_names = make_table(files, this_bin, bin_width, y, x)
#     plot_table(files, this_bin, bin_width, y, x)
    


def plot_spectra(map_name):
    # line belows causes a loop of plots, their number=this_bin. If you want just a single plot, comment the line out
    Tmax = fits.getdata(map_name)
    bin_arr = np.sort(Tmax[np.isfinite(Tmax)])[::-1]
    max_len = len(bin_arr)
    
    for bin_iteration in range(max_len):
        loop_count = 0
        fig = plt.figure(figsize=(14,10))
        # for plotting plots in a single figure put plt.figure before all the .plot commands
        # otherwise you will get the plots in a different figures

        y, x, med, bin_num = progression_binning(map_name, bin_num)
        # line below rounds med to 3 significant figures, means: 1.23456E+22 -> 1.23E+22
        med_rounded = float('%.3g' % med)

        for file_name in files:                        
            sp_av, cube = averaging(file_name, y, x)
            thiscube_spectrum_dv, cube, offset_velocity, sp_av = averaging_over_dopplervel(file_name, y, x)
            #Change to velocity axis and such here as well.  Then average all spectra with that bin label.
            ax = fig.add_subplot(len(files),1,loop_count+1)
            plt.plot(offset_velocity,thiscube_spectrum_dv, drawstyle='steps')
            plt.yticks(np.arange(min(thiscube_spectrum_dv), max(thiscube_spectrum_dv), (max(thiscube_spectrum_dv)-min(thiscube_spectrum_dv))/3)) 
            plt.xticks(np.arange(-30, 30, 5))
            ax.tick_params(axis='x', labelsize=22)
            ax.tick_params(axis='y', labelsize=20)

            # the two line below make the x-axis to be labelled with number values only for the very bottom spectrum.
            # It is a way to save space and not get overwhelmed with number on the plot.
            if file_name != files[-1]:
                plt.setp(ax.get_xticklabels(), visible=False)

            file_name = file_name.strip('L1455_').strip('_all.fits').strip('NGC1333').strip('_DR1_all.fits').strip('OrionA_').strip('_base_DR1.fits')
            plt.legend([file_name], bbox_to_anchor=(1.01, 1.15), prop={'size':20})

            loop_count += 1        

        fig.suptitle("map=%r: Averaged spectrum from \n $\overline{T}_{antena} = %rK$, progressive bin size"
                     %(map_name, med_rounded), fontsize=20)
        plt.ylabel(r'$T_{antena} (K)$', fontsize = 24)
        plt.xlabel(r'$v_{\mathrm{LSR}}\ (\mathrm{km} \cdot \mathrm{s}^{-1})$', fontsize = 24)
        # I use .pdf for savefig through out the code, because it gives more than suficient resolution
        plt.savefig("map=%r:BinsAveraging_progressive_bin_size.pdf" %(map_name))
        plt.show()
        if bin_iteration == bin_num:
            break

      
def binning(bin_width, this_bin, map_name):
    """A function creating brightness bins of pixels, and eventualy a map, in the given spectral cube"""
#     cube = SpectralCube.read(map_name)
#     cube = cube.with_spectral_unit(u.km/u.s,velocity_convention='radio')
#     Tmax = cube.apply_numpy_function(np.nanmax,axis=0) # array of the maximum values in the spectra of each pixel
#     baddata = nd.morphology.binary_dilation(np.isnan(Tmax),np.ones((25,25)))
#     Tmax[baddata]=0.0
#     Tmax[np.isfinite(Tmax)]
    
    Tmax = fits.getdata(map_name)
    bin_arr = np.sort(Tmax[np.isfinite(Tmax)])
    bin_arr2 = bin_arr[:: - bin_width] # this creates an array of the bin margins, in which every bin has a width of "bin_width"  
    bins = np.digitize(Tmax,bin_arr2)
    bins = [x-1 for x in bins] # this and the following line correct the fact that there is otherwise
    bins = np.array(bins) # only one pixel in the bin = 0
    y, x = np.where(bins==this_bin)
    med = round(np.median(Tmax[y,x]), 2)
    return y, x, med

def progression_binning(bin_iteration, map_name):
    """Analogous to the 'binning'-function, except for the bin-width changing progressively with the pixel-numbwer.
    It starts with 100pixels/bin for the brightest/densest regions and goes up to 1000 for the faintest."""
#     cube = SpectralCube.read(map_name)
#     cube = cube.with_spectral_unit(u.km/u.s,velocity_convention='radio')
#     Tmax = cube.apply_numpy_function(np.nanmax,axis=0) # array of the maximum values in the spectra of each pixel
#     baddata = nd.morphology.binary_dilation(np.isnan(Tmax),np.ones((25,25)))
#     Tmax[baddata]=0.0
#     Tmax[np.isfinite(Tmax)]
    
    Tmax = fits.getdata(map_name)
#     bin_arr = np.sort(Tmax[np.isfinite(Tmax)])
    bin_arr = np.sort(Tmax[np.isfinite(Tmax)])[::-1]
     
    # n is the number of sub arrays bin_arr is being separated to
    # I set as default n=4, since only 1/4 of all the pixels carry data good enough for analysis
    
    # In the 2 lines below we shorten bin_arr, so the number of its elements is a multiple of 1000
    a = len(bin_arr)%1600
    print 'a=', a
    bin_arr = bin_arr[:-a]
    print 'len(bin_arr)=', len(bin_arr)
    
    # next we want to tak the first 1/3 of bin_arr, that is also still a multiple of 1000
    # I choose the first 1/3, because this is where approximately the good pixel data is located
    # anything further can not be analysed properly
    b = len(bin_arr)/1600
    print 'b=', b
    bin_num = b/3
    print 'bin_num=', bin_num
    # the line below connects the elements of bin_arr into sub arrays, that are part of a larger array f(A).
    # n gives the length of the sub_arrays
    # Example: if bin_arr = [1,2,3,4,5,6] and n=2 => f(A)=[[1,2], [3,4], [5,6]]
    f = lambda bin_arr, l=(bin_num*1600): [bin_arr[i:i+l] for i in range(0, len(bin_arr), l)]
    print 'f(bin_arr)=', f(bin_arr)
    print 'len(f(bin_arr))=', len(f(bin_arr))
    print 'len(f(bin_arr)[0])=', len(f(bin_arr)[0])
    
    # Now we have to do another iteration and divide the array again in m sub-arrays.
    # Every sub array would be devided in bins, each bin cointaining 125*(2.0**i)pixels/bin
    arr_good = f(bin_arr)[0]
#     m=20
    f = lambda arr_good, l=1600: [arr_good[i:i+l] for i in range(0, len(arr_good), l)]
    
    progr_bin_arr = []
    for i in range(bin_num):
        # the line below sets the progressive bin width: bin_wi(0)=1000, bin_wi(1)=500, bin_wi(2)=250
        bin_wi = 50*(2.0**i)
        print 'len(f(arr_good[i]))=', len(f(arr_good)[i])
        bin_arr_i = f(arr_good)[i][:: bin_wi] # this creates an array of the bin margins, in which every bin has a width of "bin_wi"  
        print 'len(bin_arr_i)=', len(bin_arr_i)
        # the line below just adds bin_arr_i to progr_bin_arr
        progr_bin_arr=numpy.concatenate((progr_bin_arr, bin_arr_i))
    progr_bin_arr = np.sort(progr_bin_arr)[::-1]

    bins = np.digitize(Tmax,progr_bin_arr)
    bins = [x-1 for x in bins] # this and the following line correct the fact that there is otherwise
    bins = np.array(bins) # only one pixel in the bin = 0
    y, x = np.where(bins==bin_iteration)
    med = round(np.median(Tmax[y,x]), 2)
    print 'len(x)=', len(x)
    print 'len(y)=', len(y)
    return y, x, med, bin_num

def averaging(file_name, y, x):
    cube = SpectralCube.read(file_name)
    cube = cube.with_spectral_unit(u.km/u.s,velocity_convention='radio')
    sp_av = np.nanmean(cube.filled_data[:,y,x].value,axis=1)
    return sp_av, cube

def averaging_over_dopplervel(file_name, y, x):
    cube = SpectralCube.read(file_name)
    cube = cube.with_spectral_unit(u.km/u.s,velocity_convention='radio')
    sp_av = np.nanmean(cube.filled_data[:,y,x].value,axis=1)
    thiscube_spectrum_dv, offset_velocity = GAS.gasBinning.BinByMask(cube, CentroidMap = vlsr, x = x, y = y)
    return thiscube_spectrum_dv, cube, offset_velocity, sp_av


def make_table_and_plot(files, map_name, y, x, table_content = 'spectral_integral', **kwargs):        
    Tmax = fits.getdata(map_name)
    bin_arr = np.sort(Tmax[np.isfinite(Tmax)])[::-1]
    max_len = len(bin_arr)
    y, x, med, bin_num = progression_binning(max_len, map_name)
    
    
    if table_content == 'spectral_integral':
        table_names = ['map', 'bin width', 'bin#', 'quantity_med']
        for f in files:
            table_names.append(f.strip('NGC1333').strip('_DR1_all.fits'))
            table_names.append(f.strip('NGC1333').strip('_DR1_all.fits')+'_err')
        y_label=r'$\int T_A \mathrm{d}v$ along line of sight'
            
        arr = np.arange(len(table_names)*bin_num).reshape(bin_num, len(table_names))
        t = Table(arr, names=table_names, meta={'name': 'table'}, dtype=['S30'] + (len(table_names)-1)*['f8']) 

        for bin_iteration in range(bin_num):
            y, x, med = progression_binning(bin_width, bin_iteration, map_name)
            row = [map_name, bin_width, bin_iteration, med]
            for file_name in files:
                thiscube_spectrum_dv, cube, offset_velocity, sp_av = averaging_over_dopplervel(file_name, y, x)
                sp_integral, error_bar = spectral_integral_and_err(offset_velocity, thiscube_spectrum_dv, cube, file_name)
                row.append(sp_integral)
                row.append(error_bar)
            t[bin_iteration] = row
        
        fig = plt.figure(figsize=(14,10))
        ax1 = fig.add_subplot(111)
        ax1.set_title("Spectral integrals, hyperfine included \n Map=%r, progressive bin size"%(map_name), fontsize=26)    
        plt.xlabel(r'$\overline{T_A}$', fontsize = 26)
        plt.ylabel(r'$\int T_A \mathrm{d}v$ along line of sight', fontsize = 26)
        ax1.set_xscale('symlog')
        ax1.set_yscale('symlog', linthreshy=0.05)
        ax1.grid(b=True, which='major', color='b', linestyle='--')
        ax1.tick_params(axis='x', labelsize=22)
        ax1.tick_params(axis='y', labelsize=22)
        ax1.set_xlim([min(t['quantity_med']),max(t['quantity_med'])])
        
        transpa_factors = [[1,0.5,0.5,0.15] for i in range(len(t))]
        # loop is there for plotting only the right columns of the table (no 'error'-columns, no 'map', ect.)
        for name in table_names:
            if '_err' in name:
                continue
            if 'map' in name or 'bin width' in name or 'bin#' in name or 'quantity_med' in name:
                continue
            plt.plot(t['quantity_med'], t[name], linewidth=3.0)
            for x, y, err, transpa_factor in zip(t['quantity_med'], t[name], t[name+'_err'], transpa_factors):
                ax1.errorbar(x, y, err, marker='o', color=transpa_factor, linestyle='-', lw=2) #lw is the line width of bar # yerr=t[column_name+'_err'], label=name, 

#         for file_name in files:
#             column_name = file_name.strip('NGC1333').strip('_DR1_all.fits') 
#             ax1.errorbar(t['quantity_med'], t[column_name], yerr=t[column_name+'_err'], marker='o', linestyle='-', label=str(column_name)) 
        
        leg = ax1.legend(prop={'size':15})
        plt.savefig("Map=%r_%r_of_$NH_3$_progressive_bin_size.pdf" %(map_name, table_content))
        plt.show()
        plt.close(fig)        
    
            
    else:
        if table_content == 'chemical_abundance':
            table_names = ['map', 'bin width', 'bin#', 'log(N(H2))', 'chem. abund.', 'chem_ab_err']
            y_label=r'chem. abund. $\log\left(\frac{N(\mathrm{NH}_3)}{N(\mathrm{H}_2)}\right)$'
            arr = np.arange(len(table_names)*bin_num).reshape(bin_num, len(table_names))
            t = Table(arr, names=table_names, meta={'name': 'table'}, dtype=['S30'] + (len(table_names)-1)*['f8'])
            for bin_iteration in range(bin_num):
                y, x, med, _ = progression_binning(bin_iteration, map_name)
                # in the bottom two lines the medium values of the collumn density of this region is being obtained
                # in is equal to log(N(H_2))
#                 column_dens_value = fits.getdata(map_column_dens)  
#                 logNH2 = log10(round(np.median(column_dens_value[y,x]), 2))
                logNH2 = log10(float('%.3g' % med))
                parameter_info, error_bars = spec_curve_fit(bin_iteration, map_name)
                logNNH3 = parameter_info[2].value
                chem_abund = logNNH3 - logNH2 # = log(N(NH3)/N(H2))
                logNNH3_error_bar = error_bars[2] #needs to be corrected, see standart deviation of logarithmic calculations
                row = [map_name, bin_width, bin_iteration, logNH2, chem_abund, logNNH3_error_bar]
                t[bin_iteration] = row
                
            
        elif table_content == 'temperature_kin':
            table_names = ['map', 'bin width', 'bin#', 'log(N(H2))', 'Temp_kin', 'Temp_kin_err']
            y_label = r'$T_{kin}$'
            arr = np.arange(len(table_names)*this_bin).reshape(this_bin, len(table_names))
            t = Table(arr, names=table_names, meta={'name': 'table'}, dtype=['S30'] + (len(table_names)-1)*['f8'])
            for bin_num in range(this_bin):
                y, x, med = binning(bin_width, bin_num)
                # in the bottom two lines the medium values of the collumn density of this region is being obtained
                # in is equal to log(N(H_2))
#                 column_dens_value = fits.getdata(map_column_dens)  
#                 logNH2 = log10(round(np.median(column_dens_value[y,x]), 2))
                logNH2 = log10(float('%.3g' % med))
                parameter_info, error_bars = spec_curve_fit(bin_num)
                Temp_kin = parameter_info[0].value
                Temp_kin_error_bar = error_bars[0]
                row = [map_name, bin_width, bin_num, logNH2, Temp_kin, Temp_kin_error_bar]
                t[bin_num] = row
                
        
        elif table_content == 'line_width':
            table_names = ['map', 'bin width', 'bin#', 'log(N(H2))', 'line_width', 'line_width_err']
            y_label = r'line width $\sigma$'
            arr = np.arange(len(table_names)*this_bin).reshape(this_bin, len(table_names))
            t = Table(arr, names=table_names, meta={'name': 'table'}, dtype=['S30'] + (len(table_names)-1)*['f8'])
            for bin_num in range(this_bin):
                y, x, med = binning(bin_width, bin_num, map_name)
                # in the bottom two lines the medium values of the collumn density of this region is being obtained
                # in is equal to log(N(H_2))
#                 column_dens_value = fits.getdata(map_column_dens)  
#                 logNH2 = log10(round(np.median(column_dens_value[y,x]), 2))
                logNH2 = log10(float('%.3g' % med))
                parameter_info, error_bars = spec_curve_fit(bin_num, map_name)
                line_width = parameter_info[3].value
                line_width_error_bar = error_bars[3] #needs to be corrected, see standart deviation of logarithmic calculations
                row = [map_name, bin_width, bin_num, logNH2, line_width, line_width_error_bar]
                t[bin_num] = row
                
#     t.write('filename.txt', format='latex') # - COOL!!!
#     t.write('NGC1333_bin_width=%r_MaxBin#=%r_no_header.txt' %(bin_width, this_bin), format='ascii.no_header')
#     t.write('OrionA:Table_spec_integrals_corrected_bin_width=%r_MaxBin#=%r_HYPERFINE.txt' %(bin_width, this_bin), format='ascii.fixed_width')
    
    if table_content != 'spectral_integral':
        fig = plt.figure(figsize=(14,10))
        ax1 = fig.add_subplot(111)
        ax1.set_title("Map=%r: \n %r of $\mathrm{NH}_3$. \n Progressive bin size: 32*50px/bin, 16*100, 8*200, 4*400, 2*800, rest*1600 "%(map_name, table_content), fontsize=24) #     
        plt.xlabel(r'$\log\left(N(\mathrm{H}_2)\right)$ along line of sight', fontsize = 28) # don't forget the r infront of '...' for latex formula
        plt.ylabel(y_label, fontsize = 28)
        ax1.tick_params(axis='x', labelsize=20)
        ax1.tick_params(axis='y', labelsize=20)
        # the line below sets the y-range to be 5% above the max-value and 5% below the min-value
        ax1.set_ylim([np.amin(t[table_names[-2]]) - 0.05*np.absolute(amin(t[table_names[-2]])), np.amax(t[table_names[-2]]) + 0.05*np.absolute(amax(t[table_names[-2]]))])

        transpa_factors = [[1,0,0,0.15] for i in range(len(t))]
        plt.plot(t[table_names[-3]], t[table_names[-2]], linewidth=3.0)
        for x, y, err, transpa_factor in zip(t[table_names[-3]], t[table_names[-2]], t[table_names[-1]], transpa_factors):
            ax1.errorbar(x, y, err, marker='o', color=transpa_factor, linestyle='-', label=table_names[-2], lw=2) #lw is the line width of bar # yerr=t[column_name+'_err'],

    #             leg = ax1.legend(prop={'size':10})
        plt.savefig("Prog_bin_50*2^i__Map=%r_%r_of_$NH_3$_bins#=%r_bin_width%r.pdf" %(map_name, table_content, bin_width, this_bin))
        plt.show()
        plt.close(fig)
    
    return t, table_names
    
    
def spectral_integral_and_err(offset_velocity, thiscube_spectrum_dv, cube, file_name):
    # first, calculating the integral
    lines_margins = {'NH3_11': [[-1.5, 1.5],[6.75, 8.5],[18.75,20.], [-8.5,-7.], [-20.5, -19.25]], 'NH3_22': [[-1.5, 1.5], [-17.5, -15.5], [15., 17.]],
                 'NH3_33': [[-1.5, 1.5]], 'C2S': [[-1.5, 1.5]], 'HC5N': [[-1.5, 1.5]], 'HC7N_21_20': [[-1.5, 1.5]],
                    'HC7N_22_21': [[-1.5, 3.]]}
    noise_margins = {'NH3_11': [12., 17.], 'NH3_22': [5., 12.5], 'NH3_33': [-12, -10], 'C2S': [-12, -10], 'HC5N': [10., 27.5], 'HC7N_21_20': [5., 15.],
                    'HC7N_22_21': [5., 15.]}
    
    if 'NH3_11' in file_name:
        file_name = 'NH3_11'
    elif 'NH3_22' in file_name:
        file_name = 'NH3_22'
    elif 'NH3_33' in file_name:
        file_name = 'NH3_33'
    elif 'C2S' in file_name:
        file_name = 'C2S'
    elif 'HC5N' in file_name:
        file_name = 'HC5N'
    elif 'HC7N_21_20' in file_name:
        file_name = 'HC7N_21_20'
    elif 'HC7N_22_21' in file_name:
        file_name = 'HC7N_22_21'
        
        
    # in line below calculating the spectral-bin-width, assuming that all spectral-bins have the same width
    spec_bin_w = (cube.spectral_axis[1] - cube.spectral_axis[2])
    sp_integral = 0

    for line_interval in lines_margins[file_name]:
        ind_line = (line_interval[0] < offset_velocity) & (offset_velocity < line_interval[1])
        sp_integral += np.nansum(thiscube_spectrum_dv[ind_line]) * spec_bin_w

# below calculating the error, based on the standard deviation of the signal noise
        ind_offset = (noise_margins[file_name][0] < offset_velocity) & (offset_velocity < noise_margins[file_name][1])
        std_noise = np.nanstd(thiscube_spectrum_dv[ind_offset])
        # Nchan should be the # of channels, that contribute to the integral (= # of data points in spectral line)
        Nchan = len(thiscube_spectrum_dv[ind_line])
        error = std_noise*np.sqrt(Nchan)
     
    return sp_integral.value, error
    
def spec_curve_fit(bin_num, map_name):
    # following loop has not good style. One should build in some break statements or error messages
    # if files repeatedly appear in loop.
    
    Tmax = fits.getdata(map_name)
    bin_arr = np.sort(Tmax[np.isfinite(Tmax)])[::-1]
    max_len = len(bin_arr)
    
    for one_file in files:
        if 'NH3_11' in one_file:
            file_name_NH3_11 = one_file
        if 'NH3_22' in one_file:
            file_name_NH3_22 = one_file
        if 'NH3_33' in one_file:
            file_name_NH3_33 = one_file
        
    y, x, med, _ = progression_binning(max_len, map_name)

    s11, _, offset_velocity11, sp_av11 = averaging_over_dopplervel(file_name_NH3_11, y, x)
    s22, _, offset_velocity22, sp_av22 = averaging_over_dopplervel(file_name_NH3_22, y, x)
    s33, _, offset_velocity33, sp_av33 = averaging_over_dopplervel(file_name_NH3_33, y, x)
    
    xarr11 = SpectroscopicAxis(offset_velocity11*u.km/u.s, velocity_convention='radio',
                           refX=freq_dict['oneone']).as_unit(u.GHz)
    xarr22 = SpectroscopicAxis(offset_velocity22*u.km/u.s, velocity_convention='radio',
                           refX=freq_dict['twotwo']).as_unit(u.GHz)
    xarr33 = SpectroscopicAxis(offset_velocity33*u.km/u.s, velocity_convention='radio',
                           refX=freq_dict['threethree']).as_unit(u.GHz)
        
    sp11 = psk.Spectrum(data=s11, xarr=xarr11, xarrkwargs={'unit':'km/s'},unit='K')
    sp22 = psk.Spectrum(data=s22, xarr=xarr22, xarrkwargs={'unit':'km/s'},unit='K')
    sp33 = psk.Spectrum(data=s33, xarr=xarr33, xarrkwargs={'unit':'km/s'},unit='K')
    
    # This joins all the spectra together into one object.
    allspec = psk.Spectra([sp11,sp22,sp33])
    allspec.xarr.as_unit('Hz',velocity_convention='radio')
    # This add the cold_ammonia model to the list of things we can use for fitting
    allspec.specfit.Registry.add_fitter('cold_ammonia',ammonia.cold_ammonia_model(),6)
    # This does the fit.  The values of the guess are 
    # Kinetic Temperature (usually about 15 to 25 K)
    # Excitation Temperature (between 2.73 K and the excitation temperature)
    # Log Column Density of ammonia
    # Line width (~1 km/s)
    # Offset velocity (you will usually use 0 instead of 8.5)
    # Ortho fraction (leave at 0)
    allspec.specfit(fittype='cold_ammonia',guesses=[23,5,13.1,1,0,0])
    
    # You can make a plot here.
    fig = plt.figure()
    allspec.plotter()
    allspec.specfit.plot_fit(lw=1, components=True)
    plt.xlim((23.692,23.697))
#     plt.xlim((23.72,23.725))
#     plt.xlim((23.8692, 23.8708))
#     plt.savefig("OrionA:pyspeckit_fit_bin_width=%rthis_bin=%r.ps" %(bin_width, this_bin))
#     plt.savefig("Map=%r:bin_num=%r_pyspeckit_fit_NH3_11TEST.eps"%(map_name, bin_num))
    plt.show()
    plt.close(fig)
    
    # returns the values of the fitted parameters: T_K, T_ex, N, sigma, v, F_0
    return allspec.specfit.parinfo, allspec.specfit.parinfo.errors
    
main(map_name)


Using matplotlib backend: Qt4Agg
Populating the interactive namespace from numpy and matplotlib
a= 64
len(bin_arr)= 65600
b= 41
bin_num= 13
f(bin_arr)= [array([  7.42259303e+22,   7.30171752e+22,   7.26586133e+22, ...,
         1.83380933e+21,   1.83375679e+21,   1.83333765e+21]), array([  1.83326975e+21,   1.83325930e+21,   1.83313154e+21, ...,
         1.03064421e+21,   1.03062694e+21,   1.03062206e+21]), array([  1.03061965e+21,   1.03059606e+21,   1.03059166e+21, ...,
         6.15158216e+20,   6.15154201e+20,   6.14973079e+20]), array([  6.14940757e+20,   6.14921393e+20,   6.14896562e+20, ...,
         4.72954737e+20,   4.72425565e+20,   4.72396713e+20])]
len(f(bin_arr))= 4
len(f(bin_arr)[0])= 20800
len(f(arr_good[i]))= 1600
len(bin_arr_i)= 32
len(f(arr_good[i]))= 1600
len(bin_arr_i)= 16
len(f(arr_good[i]))= 1600
len(bin_arr_i)= 8
len(f(arr_good[i]))= 1600
len(bin_arr_i)= 4
len(f(arr_good[i]))= 1600
len(bin_arr_i)= 2
len(f(arr_good[i]))= 1600
len(bin_arr_i)= 1
len(f(arr_good[i]))=

using a non-integer number instead of an integer will result in an error in the future
Mean of empty slice


ValueError: Data are invalid; cannot be fit.

In [113]:
transpa_factors = [[1,0,0,0.2] for i in range(3)]
for x, factor in zip([10,20,30], transpa_factors):
    print x, factor

10 [1, 0, 0, 0.2]
20 [1, 0, 0, 0.2]
30 [1, 0, 0, 0.2]


In [14]:
table_names = ['map', 'bin width', 'bin#', 'quantity_med', 'bla' ,'1', '2', '3', '3_err']
for name in table_names:
    if '_err' in name:
        continue
    if 'map' in name or 'bin width' in name or 'bin#' in name or 'quantity_med' in name:
        continue
    print name
#     if not '_err' in name and not name in ('map' and 'bin_width' and 'bin#' and 'quantity_med'):
#         print name

bla
1
2
3


In [1]:
A = [i for i in range(100)]
n = len(A)/3
f = lambda A, n= len(A)/3: [A[i:i+n] for i in range(0, len(A), n)]
print f(A)

[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32], [33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65], [66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98], [99]]


In [21]:
bin_arr = range(500)
print 'initial bin_arr2=', bin_arr[:: - 15]

n=4
f = lambda bin_arr, l=len(bin_arr)/n: [bin_arr[i:i+l] for i in range(0, len(bin_arr), l)] 

print 'len(f(bin_arr))=', len(f(bin_arr))
print 'f(bin_arr)[4]=', f(bin_arr)[3]

progr_bin_arr = []
for i in range(n):
    bin_wi = 15*(i+1)
    bin_arr_i = ('bin_arr_' + str(i))
    bin_arr_i = f(bin_arr)[i][:: - bin_wi]
    print 'bin_arr_i', bin_arr_i
    progr_bin_arr += bin_arr_i

print 'progr_bin_arr=', progr_bin_arr 

initial bin_arr2= [499, 484, 469, 454, 439, 424, 409, 394, 379, 364, 349, 334, 319, 304, 289, 274, 259, 244, 229, 214, 199, 184, 169, 154, 139, 124, 109, 94, 79, 64, 49, 34, 19, 4]
len(f(bin_arr))= 4
f(bin_arr)[4]= [375, 376, 377, 378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 389, 390, 391, 392, 393, 394, 395, 396, 397, 398, 399, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431, 432, 433, 434, 435, 436, 437, 438, 439, 440, 441, 442, 443, 444, 445, 446, 447, 448, 449, 450, 451, 452, 453, 454, 455, 456, 457, 458, 459, 460, 461, 462, 463, 464, 465, 466, 467, 468, 469, 470, 471, 472, 473, 474, 475, 476, 477, 478, 479, 480, 481, 482, 483, 484, 485, 486, 487, 488, 489, 490, 491, 492, 493, 494, 495, 496, 497, 498, 499]
bin_arr_i [124, 109, 94, 79, 64, 49, 34, 19, 4]
bin_arr_i [249, 219, 189, 159, 129]
bin_arr_i [374, 329, 284]
bin_arr_i [499, 439, 379]
progr_bin_arr= [124, 10

In [10]:
bin_arr = range(65345)
a = len(bin_arr)%1000
print 'a=', a
bin_arr = bin_arr[:-a]
print 'len(bin_arr)=', len(bin_arr)

# next we want to tak the first 1/3 of bin_arr, that is also still a multiple of 1000
# I choose the first 1/3, because this is where approximately the good pixel data is located
# anything further can not be analysed properly
b = len(bin_arr)/1000
print 'b=', b
c = b/3
print 'c=', c
# the line below connects the elements of bin_arr into sub arrays, that are part of a larger array f(A).
# n gives the length of the sub_arrays
# Example: if bin_arr = [1,2,3,4,5,6] and n=2 => f(A)=[[1,2], [3,4], [5,6]]
f = lambda bin_arr, l=(c*1000): [bin_arr[i:i+l] for i in range(0, len(bin_arr), l)]
# print 'f(bin_arr)=', f(bin_arr)
print 'len(f(bin_arr))=', len(f(bin_arr))
print 'len(f(bin_arr)[0])=', len(f(bin_arr)[0])

# Now we have to do another iteration and divide the array again in m sub-arrays.
# Every sub array would be devided in bins, each bin cointaining 125*(2.0**i)pixels/bin
arr_good = f(bin_arr)[0]
# m=20
f = lambda arr_good, l=1000: [arr_good[i:i+l] for i in range(0, len(arr_good), l)]

progr_bin_arr = []
for i in range(m):
    # the line below sets the progressive bin width: bin_wi(0)=1000, bin_wi(1)=500, bin_wi(2)=250
    bin_wi = 125*(2**i)
    print 'len(f(arr_good)[i])=', len(f(arr_good)[i])
    print 'bin_wi =', bin_wi
    bin_arr_i = f(arr_good)[i][:: bin_wi] # this creates an array of the bin margins, in which every bin has a width of "bin_wi"  
    print 'len(bin_arr_i)=', len(bin_arr_i)
    # the line below just adds bin_arr_i to progr_bin_arr
    progr_bin_arr=numpy.concatenate((progr_bin_arr, bin_arr_i))
progr_bin_arr = np.sort(progr_bin_arr)[::-1]

a= 345
len(bin_arr)= 65000
b= 65
c= 21
len(f(bin_arr))= 4
len(f(bin_arr)[0])= 21000
len(f(arr_good)[i])= 1000
bin_wi = 125
len(bin_arr_i)= 8
len(f(arr_good)[i])= 1000
bin_wi = 250
len(bin_arr_i)= 4
len(f(arr_good)[i])= 1000
bin_wi = 500
len(bin_arr_i)= 2
len(f(arr_good)[i])= 1000
bin_wi = 1000
len(bin_arr_i)= 1
len(f(arr_good)[i])= 1000
bin_wi = 2000
len(bin_arr_i)= 1
len(f(arr_good)[i])= 1000
bin_wi = 4000
len(bin_arr_i)= 1
len(f(arr_good)[i])= 1000
bin_wi = 8000
len(bin_arr_i)= 1
len(f(arr_good)[i])= 1000
bin_wi = 16000
len(bin_arr_i)= 1
len(f(arr_good)[i])= 1000
bin_wi = 32000
len(bin_arr_i)= 1
len(f(arr_good)[i])= 1000
bin_wi = 64000
len(bin_arr_i)= 1
len(f(arr_good)[i])= 1000
bin_wi = 128000
len(bin_arr_i)= 1
len(f(arr_good)[i])= 1000
bin_wi = 256000
len(bin_arr_i)= 1
len(f(arr_good)[i])= 1000
bin_wi = 512000
len(bin_arr_i)= 1
len(f(arr_good)[i])= 1000
bin_wi = 1024000
len(bin_arr_i)= 1
len(f(arr_good)[i])= 1000
bin_wi = 2048000
len(bin_arr_i)= 1
len(f(arr_good)[i])= 1000
bin_wi =